# DWD_historical_weather: Beispiel-Notebook

#### Bundesland als globalen Parameter festlegen

In [1]:
BUNDESLAND = 'Berlin'

In [2]:
from DWD_hist_weather import tagestemp_land, tageswerte_land

import pandas as pd
import pickle

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

#### Das eigentliche Einlesen der Daten: Wenn vorhanden aus pickle, sonst **tageswerte_land** aus dem Modul aufrufen und die Daten vom DWD ziehen

In [3]:
pickle_dateiname = f'{BUNDESLAND}.pickle'
try:
    tageswerte = pickle.load(open(pickle_dateiname, 'rb'))
    print(f'Wetterdaten für {BUNDESLAND} aus pickle eingelesen.')
except (OSError, IOError):
    tageswerte = tageswerte_land(BUNDESLAND)
    pickle.dump(tageswerte, open(pickle_dateiname, 'wb'))
    print(f'\nWetterdaten für {BUNDESLAND} in pickle geschrieben.')

.-.-...-...-.-.-...-.-...-.-.-.-...-...-.-
Wetterdaten für Berlin in pickle geschrieben.


### DataFrame ausgeben

In [4]:
display(tageswerte)

,TempMean,HumidityMean,TempMax,TempMin,SunshineDuration,Windspeed,Jahr,Monat,Tag_des_Jahres
Datum,,,,,,,,,
1876-01-01,2.200000,NaN,3.500000,-3.000,NaN,NaN,1876,1,1
1876-01-02,2.500000,NaN,2.800000,2.300,NaN,NaN,1876,1,2
1876-01-03,0.300000,NaN,2.800000,-1.800,NaN,NaN,1876,1,3
1876-01-04,-5.800000,NaN,-1.800000,-7.000,NaN,NaN,1876,1,4
1876-01-05,-9.800000,NaN,-6.800000,-13.300,NaN,NaN,1876,1,5
...,...,...,...,...,...,...,...,...,...
2021-12-17,5.275000,86.157500,6.000000,4.575,0.0000,5.0,2021,12,351
2021-12-18,5.825000,92.417500,7.675000,4.700,0.0000,4.3,2021,12,352
2021-12-19,5.900000,87.325000,7.275000,1.125,0.0000,6.9,2021,12,353


### Heatmap der täglichen Durchschnittstemperaturen

In [ ]:
ana = tageswerte.pivot(index='Jahr', columns='Tag_des_Jahres', values='TempMean')

f, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(ana, vmin=-10, vmax=23, cmap="RdBu_r")
ax.axes.set_title("Tagesmitteltemperaturen", y=1.01)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

### Jährliche Durchschnittstemperaturen plus 5-Jahres-Mittel

In [ ]:
ana = tageswerte.pivot(index='Jahr', columns='Tag_des_Jahres', values='TempMean')

ana['Jahresmittel'] = ana.mean(axis=1)
ana['Jahresmittel5'] = ana['Jahresmittel'].rolling(5).mean()

plt.subplots(figsize=(20, 10))
sns.lineplot(data=ana, x='Jahr', y='Jahresmittel')
sns.lineplot(data=ana, x='Jahr', y='Jahresmittel5', color='red')